In [75]:
import pandas as pd 
import numpy as np 
import os

from utils import * 

df_2012_summer = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "summer.csv")))
df_2012_winter = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "winter.csv")))
df_2012 = pd.concat([df_2012_summer, df_2012_winter])
df_2012 = df_2012.drop('Athlete', axis=1)
df_2016 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2016, "athletes.csv")))
df_2020 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2020, "medals.csv")))


df_2016 = df_2016.rename({'nationality': 'country_code', 'gold': 'gold_medal', 'silver': 'silver_medal', 'bronze': 'bronze_medal'}, axis=1)
df_2016 = df_2016[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
df_2016 = df_2016.groupby('country_code').agg(sum)
df_2016 = df_2016.reset_index(drop=False)
df_2016.insert(0, 'year', 2016)
df_2016['total'] = df_2016['gold_medal'] + df_2016['silver_medal'] + df_2016['bronze_medal']

df_2020['gold_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==1 else 0)
df_2020['silver_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==2 else 0)
df_2020['bronze_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==3 else 0)
# df_2020['year'] = 2020
df_2020 = df_2020[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
df_2020 = df_2020.groupby('country_code').agg(sum)
df_2020 = df_2020.reset_index(drop=False)
df_2020.insert(0, 'year', 2020)
df_2020['total'] = df_2020['gold_medal'] + df_2020['silver_medal'] + df_2020['bronze_medal']

# print(f'df_2012:\n{df_2012.head()} \n df_2016:\n{df_2016.head()} \n df_2020:\n{df_2020.head()}')

In [76]:
df_2016.head()

,year,country_code,gold_medal,silver_medal,bronze_medal,total
0,2016,AFG,0,0,0,0
1,2016,ALB,0,0,0,0
2,2016,ALG,0,2,0,2
3,2016,AND,0,0,0,0
4,2016,ANG,0,0,0,0


In [77]:
df_2020.head()

,year,country_code,gold_medal,silver_medal,bronze_medal,total
0,2020,ARG,0,18,25,43
1,2020,ARM,0,2,2,4
2,2020,AUS,36,27,66,129
3,2020,AUT,1,1,5,7
4,2020,AZE,0,3,4,7


# year, country, gold_medal, silver_medal, bronze_medal, total

In [72]:
df_2012 = df_2012[df_2012.Year>= 1948]
df_2012 = df_2012.reset_index()
df_2012.head()

,index,Year,City,Sport,Discipline,Country,Gender,Event,Medal
0,7204,1948,London,Aquatics,Diving,MEX,Men,10M Platform,Bronze
1,7205,1948,London,Aquatics,Diving,USA,Men,10M Platform,Gold
2,7206,1948,London,Aquatics,Diving,USA,Men,10M Platform,Silver
3,7207,1948,London,Aquatics,Diving,DEN,Women,10M Platform,Bronze
4,7208,1948,London,Aquatics,Diving,USA,Women,10M Platform,Gold
